In [305]:
import numpy as np
import pandas as pd

# Generate 10 random locations in a city grid (x, y coordinates)
np.random.seed(42)
locations = pd.DataFrame({
    "Location_ID": range(1, 11),
    "X_Coordinate": np.random.uniform(0, 100, 10),
    "Y_Coordinate": np.random.uniform(0, 100, 10)
})
locations

,Location_ID,X_Coordinate,Y_Coordinate
0,1,37.454012,2.058449
1,2,95.071431,96.990985
2,3,73.199394,83.244264
3,4,59.865848,21.233911
4,5,15.601864,18.182497
5,6,15.599452,18.340451
6,7,5.808361,30.424224
7,8,86.617615,52.475643
8,9,60.111501,43.194502
9,10,70.807258,29.122914


In [306]:

from scipy.spatial.distance import cdist

# Compute pairwise Euclidean distances
coordinates = locations[["X_Coordinate", "Y_Coordinate"]].values
distance_matrix = cdist(coordinates, coordinates, metric="euclidean")

# Convert to DataFrame for readability
distance_df = pd.DataFrame(distance_matrix, columns=locations["Location_ID"], index=locations["Location_ID"])
distance_df

Location_ID,1,2,3,4,5,6,7,8,9,10
Location_ID,,,,,,,,,,
1,0.000000,111.049328,88.706645,29.495572,27.156975,27.252988,42.497816,70.419836,46.963141,42.952581
2,111.049328,0.000000,25.833279,83.537820,111.920462,111.811010,111.350928,45.310955,64.158073,72.075136
3,88.706645,25.833279,0.000000,63.427654,86.893665,86.777061,85.624225,33.567196,42.134029,54.174190
4,29.495572,83.537820,63.427654,0.000000,44.369037,44.360861,54.833145,41.130315,21.961965,13.488914
5,27.156975,111.920462,86.893665,44.369037,0.000000,0.157973,15.677136,78.862264,51.055932,56.279021
6,27.252988,111.811010,86.777061,44.360861,0.157973,0.000000,15.552589,78.795880,50.980843,56.250896
7,42.497816,111.350928,85.624225,54.833145,15.677136,15.552589,0.000000,83.763957,55.784505,65.011922
8,70.419836,45.310955,33.567196,41.130315,78.862264,78.795880,83.763957,0.000000,28.084046,28.201371
9,46.963141,64.158073,42.134029,21.961965,51.055932,50.980843,55.784505,28.084046,0.000000,17.675090


In [307]:
# Generate random bin attributes
bins = pd.DataFrame({
    "Location_ID": range(1, 11),
    "Waste_Generation_Rate": np.random.uniform(5, 20, size=10), # kg/day
    "Bin_Capacity": np.random.uniform(50, 100, size=10),        # kg
    "Current_Fill_Level": np.random.uniform(30, 90, size=10)    # %
})

bins["Waste_loads"] = bins["Bin_Capacity"] * bins["Current_Fill_Level"] / 100
bins


,Location_ID,Waste_Generation_Rate,Bin_Capacity,Current_Fill_Level,Waste_loads
0,1,14.177793,80.377243,37.322294,29.998631
1,2,7.092408,58.526206,59.710615,34.946357
2,3,9.382170,53.252580,32.063311,17.074540
3,4,10.495428,97.444277,84.559224,82.398124
4,5,11.841050,98.281602,45.526799,44.744467
5,6,16.777639,90.419867,69.751337,63.069066
6,7,7.995107,65.230688,48.702665,31.769083
7,8,12.713517,54.883606,61.204081,33.591007
8,9,13.886219,84.211651,62.802617,52.887121
9,10,5.696756,72.007625,41.091267,29.588846


In [308]:
# Merge location and bin data
dataset = pd.merge(locations, bins, on="Location_ID")

dataset


,Location_ID,X_Coordinate,Y_Coordinate,Waste_Generation_Rate,Bin_Capacity,Current_Fill_Level,Waste_loads
0,1,37.454012,2.058449,14.177793,80.377243,37.322294,29.998631
1,2,95.071431,96.990985,7.092408,58.526206,59.710615,34.946357
2,3,73.199394,83.244264,9.382170,53.252580,32.063311,17.074540
3,4,59.865848,21.233911,10.495428,97.444277,84.559224,82.398124
4,5,15.601864,18.182497,11.841050,98.281602,45.526799,44.744467
5,6,15.599452,18.340451,16.777639,90.419867,69.751337,63.069066
6,7,5.808361,30.424224,7.995107,65.230688,48.702665,31.769083
7,8,86.617615,52.475643,12.713517,54.883606,61.204081,33.591007
8,9,60.111501,43.194502,13.886219,84.211651,62.802617,52.887121
9,10,70.807258,29.122914,5.696756,72.007625,41.091267,29.588846


In [309]:
depot = {"X_Coordinate": 0, "Y_Coordinate": 0}

def calculate_route_cost(bin_x, bin_y):
    distance_to_depot = np.sqrt((bin_x - depot["X_Coordinate"])**2 + (bin_y - depot["Y_Coordinate"])**2)
    return 2 * distance_to_depot  # Round trip

# Apply to all bins
dataset["Route_Cost"] = dataset.apply(
    lambda row: calculate_route_cost(row["X_Coordinate"], row["Y_Coordinate"]), axis=1
)
dataset

,Location_ID,X_Coordinate,Y_Coordinate,Waste_Generation_Rate,Bin_Capacity,Current_Fill_Level,Waste_loads,Route_Cost
0,1,37.454012,2.058449,14.177793,80.377243,37.322294,29.998631,75.021070
1,2,95.071431,96.990985,7.092408,58.526206,59.710615,34.946357,271.630839
2,3,73.199394,83.244264,9.382170,53.252580,32.063311,17.074540,221.700328
3,4,59.865848,21.233911,10.495428,97.444277,84.559224,82.398124,127.040132
4,5,15.601864,18.182497,11.841050,98.281602,45.526799,44.744467,47.917485
5,6,15.599452,18.340451,16.777639,90.419867,69.751337,63.069066,48.154545
6,7,5.808361,30.424224,7.995107,65.230688,48.702665,31.769083,61.947413
7,8,86.617615,52.475643,12.713517,54.883606,61.204081,33.591007,202.546827
8,9,60.111501,43.194502,13.886219,84.211651,62.802617,52.887121,148.042664
9,10,70.807258,29.122914,5.696756,72.007625,41.091267,29.588846,153.124941


In [310]:
initial_routes = pd.DataFrame({
    "Route_ID": [f"R{i}" for i in range(1, 11)],
    "Bins_Covered": [{i} for i in range(1, 11)],
    "Cost": dataset["Route_Cost"].values
})

# Create coverage matrix
for i in range(1, 11):
    initial_routes[f"a_{i}"] = initial_routes["Bins_Covered"].apply(lambda x: 1 if i in x else 0)
initial_routes

,Route_ID,Bins_Covered,Cost,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10
0,R1,{1},75.021070,1,0,0,0,0,0,0,0,0,0
1,R2,{2},271.630839,0,1,0,0,0,0,0,0,0,0
2,R3,{3},221.700328,0,0,1,0,0,0,0,0,0,0
3,R4,{4},127.040132,0,0,0,1,0,0,0,0,0,0
4,R5,{5},47.917485,0,0,0,0,1,0,0,0,0,0
5,R6,{6},48.154545,0,0,0,0,0,1,0,0,0,0
6,R7,{7},61.947413,0,0,0,0,0,0,1,0,0,0
7,R8,{8},202.546827,0,0,0,0,0,0,0,1,0,0
8,R9,{9},148.042664,0,0,0,0,0,0,0,0,1,0
9,R10,{10},153.124941,0,0,0,0,0,0,0,0,0,1


In [311]:
import pulp

# Initialize LP problem
rmp = pulp.LpProblem("Restricted_Master_Problem", pulp.LpMinimize)

# Variables
lambda_s = pulp.LpVariable.dicts("Route", initial_routes["Route_ID"], lowBound=0)

# Objective
rmp += pulp.lpSum([lambda_s[r] * initial_routes.loc[initial_routes["Route_ID"] == r, "Cost"].values[0] 
                   for r in initial_routes["Route_ID"]])

# Constraints
for i in range(1, 11):
    rmp += pulp.lpSum([lambda_s[r] * initial_routes.loc[initial_routes["Route_ID"] == r, f"a_{i}"].values[0] 
                       for r in initial_routes["Route_ID"]]) == 1, f"Coverage_Bin_{i}"

# Solve
rmp.solve()
print("Status:", pulp.LpStatus[rmp.status])
print("Total Cost:", pulp.value(rmp.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mallikarjun/research_project/muncipal_route_column_generation/muncipal/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/862ebc05f7d74d71a415898595ac8b35-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/862ebc05f7d74d71a415898595ac8b35-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 36 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 10 rows, 10 columns and 10 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-10) rows, 0 (-10) columns and 0 (-10) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 1357.1262
After Postsolve, objective 1357.1262, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1357.126242 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to a

In [312]:
selected_routes = []
for route_id in initial_routes["Route_ID"]:
    var = lambda_s[route_id]
    if var.varValue > 1e-5:  # Account for numerical precision
        selected_routes.append({
            "Route_ID": route_id,
            "Bins_Covered": initial_routes[initial_routes["Route_ID"] == route_id]["Bins_Covered"].values[0],
            "Cost": initial_routes[initial_routes["Route_ID"] == route_id]["Cost"].values[0],
            "Lambda": var.varValue
        })

# Convert to DataFrame for readability
selected_df = pd.DataFrame(selected_routes)
selected_df


,Route_ID,Bins_Covered,Cost,Lambda
0,R1,{1},75.021070,1.0
1,R2,{2},271.630839,1.0
2,R3,{3},221.700328,1.0
3,R4,{4},127.040132,1.0
4,R5,{5},47.917485,1.0
5,R6,{6},48.154545,1.0
6,R7,{7},61.947413,1.0
7,R8,{8},202.546827,1.0
8,R9,{9},148.042664,1.0
9,R10,{10},153.124941,1.0


In [313]:
initial_routes

,Route_ID,Bins_Covered,Cost,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10
0,R1,{1},75.021070,1,0,0,0,0,0,0,0,0,0
1,R2,{2},271.630839,0,1,0,0,0,0,0,0,0,0
2,R3,{3},221.700328,0,0,1,0,0,0,0,0,0,0
3,R4,{4},127.040132,0,0,0,1,0,0,0,0,0,0
4,R5,{5},47.917485,0,0,0,0,1,0,0,0,0,0
5,R6,{6},48.154545,0,0,0,0,0,1,0,0,0,0
6,R7,{7},61.947413,0,0,0,0,0,0,1,0,0,0
7,R8,{8},202.546827,0,0,0,0,0,0,0,1,0,0
8,R9,{9},148.042664,0,0,0,0,0,0,0,0,1,0
9,R10,{10},153.124941,0,0,0,0,0,0,0,0,0,1


In [314]:
def build_pricing_problem(n_bins, distance_matrix, dual_values, truck_capacity, waste_loads):
    model = pulp.LpProblem("Pricing_Problem", pulp.LpMinimize)
    
    # Variables (indices 0-10)
    nodes = range(n_bins + 1)  # 0 (depot) to 10 (bins)
    x = pulp.LpVariable.dicts("x", [(i, j) for i in nodes for j in nodes if i != j], cat='Binary')
    y = pulp.LpVariable.dicts("y", [i for i in nodes[1:]], cat='Binary')  # Bins 1-10
    
    # Objective: Minimize reduced cost
    model += (
        pulp.lpSum(distance_matrix[i][j] * x[i,j] for i in nodes for j in nodes if i != j) -
        pulp.lpSum(dual_values[i] * y[i] for i in nodes[1:])
    )
    
    # Constraints
    # 1. Start and end at depot (node 0)
    model += pulp.lpSum(x[0, j] for j in nodes[1:]) == 1  # Depart depot
    model += pulp.lpSum(x[i, 0] for i in nodes[1:]) == 1  # Return to depot
    
    # 2. Flow conservation
    for k in nodes[1:]:
        model += (
            pulp.lpSum(x[i, k] for i in nodes if i != k) == 
            pulp.lpSum(x[k, j] for j in nodes if j != k)
        )
    
    # 3. Link x and y variables
    for i in nodes[1:]:
        model += y[i] == pulp.lpSum(x[i, j] for j in nodes if j != i)
    
    # 4. Capacity constraint
    model += (
        pulp.lpSum(waste_loads[i] * y[i] for i in nodes[1:]) <= truck_capacity
    )

    # 5. Subtour elimination (MTZ formulation)
    u = pulp.LpVariable.dicts("u", nodes[1:], lowBound=1, upBound=n_bins, cat='Integer')
    for i in nodes[1:]:
        for j in nodes[1:]:
            if i != j:
                model += u[i] - u[j] + n_bins * x[i,j] <= n_bins - 1
                
    return model, x, y


def solve_pricing(model):
    model.solve(pulp.PULP_CBC_CMD(msg=1))  # Use CBC solver quietly
    if model.status == pulp.LpStatusOptimal:
        reduced_cost = pulp.value(model.objective)
        return reduced_cost
    else:
        return None

def extract_new_route(x_vars, n_bins):
    route = []
    current = 0  # Start at depot
    while True:
        next_node = None
        # Iterate over ALL nodes (0-10) and skip j == current
        for j in range(n_bins + 1):  
            if j == current:
                continue  # Skip self-node
            if x_vars[current, j].value() > 0.5:
                next_node = j
                break
        if next_node is None or next_node == 0:
            break
        route.append(next_node)
        current = next_node
    return route


In [315]:
truck_capacity = 200  # kg
# From your dataset DataFrame
waste_loads = {
    row["Location_ID"]: row["Bin_Capacity"] * (row["Current_Fill_Level"] / 100)
    for _, row in dataset.iterrows()
}

n_bins = len(waste_loads)

# Add depot (node 0) to locations
depot_row = pd.DataFrame({"Location_ID": [0], "X_Coordinate": [0], "Y_Coordinate": [0]})
locations_with_depot = pd.concat([depot_row, locations], ignore_index=True)

# Recompute distances with depot
coordinates_with_depot = locations_with_depot[["X_Coordinate", "Y_Coordinate"]].values
distance_matrix_with_depot = cdist(coordinates_with_depot, coordinates_with_depot, metric="euclidean")


In [316]:
# Correct the reduced cost calculation in the pricing problem
def pricing_problem(n_bins, distance_matrix, dual_values, truck_capacity, waste_loads):
    model, x, y = build_pricing_problem(n_bins, distance_matrix, dual_values, truck_capacity, waste_loads)
    reduced_cost = solve_pricing(model)
    if reduced_cost is None or reduced_cost >= -1e-5:
        return None
    new_route = extract_new_route(x, n_bins)
    route_cost = sum(distance_matrix[i][j] * x[i, j].value() for i in range(n_bins + 1) for j in range(n_bins + 1) if i != j)
    reduced_cost = route_cost - sum(dual_values[i] for i in new_route)
    return [{
        "route": new_route,
        "cost": route_cost,
        "reduced_cost": reduced_cost
    }]


# Add new routes to the RMP correctly
while True:
    # Solve RMP and get dual values
    rmp.solve()
    duals = {i: rmp.constraints[f"Coverage_Bin_{i}"].pi for i in range(1, 11)}

    # Generate new routes
    new_routes = pricing_problem(n_bins, distance_matrix_with_depot, duals, truck_capacity, waste_loads)
    if not new_routes:
        break
    print("New Route:", new_routes[0])

    # Add new columns to RMP
    for route in new_routes:
        route_id = f"R{len(initial_routes)+1}"
        
        # Create column variable
        col = pulp.LpVariable(route_id, lowBound=0)
        
        # Add to objective
        rmp.objective += col * route["cost"]
        
        # Update coverage constraints
        for bin in route["route"]:
            rmp.constraints[f"Coverage_Bin_{bin}"] += col
        
        # Track in DataFrame
        new_row = {"Route_ID": route_id, "Cost": route["cost"], "Bins_Covered": set(route["route"])}
        new_row.update({f"a_{i}": 1 if i in route["route"] else 0 for i in range(1, 11)})
        initial_routes = pd.concat([initial_routes, pd.DataFrame([new_row])], ignore_index=True)



# Solve the final RMP
rmp.solve()
print("Status:", pulp.LpStatus[rmp.status])
print("Total Cost:", pulp.value(rmp.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mallikarjun/research_project/muncipal_route_column_generation/muncipal/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/44b94379b59e472d881648454989f9e4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/44b94379b59e472d881648454989f9e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 36 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 10 rows, 10 columns and 10 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-10) rows, 0 (-10) columns and 0 (-10) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 1357.1262
After Postsolve, objective 1357.1262, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1357.126242 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to a

In [317]:
# After solving the final RMP
print("\n=== Optimal Routes ===")
total_cost = 0
selected_count = 0

for var in rmp.variables():
    if var.varValue > 1e-5:  # Account for numerical precision
        route_id = var.name.replace("Route_", "")
        route_data = initial_routes[initial_routes["Route_ID"] == route_id].iloc[0]
        
        bins_covered = [i for i in range(1, 11) if route_data[f"a_{i}"] == 1]
        cost = route_data["Cost"]
        
        print(f"Route {route_id}:")
        print(f"  Bins: {sorted(bins_covered)}")
        print(f"  Cost: {cost:.2f} km")
        print(f"  Waste Load: {sum(waste_loads[i] for i in bins_covered):.1f} kg")
        print("-" * 30)
        
        total_cost += cost * var.varValue
        selected_count += 1

print(f"\nTotal Routes Selected: {selected_count}")
print(f"Total Distance: {total_cost:.2f} km")
print(f"Truck Capacity Used: {truck_capacity} kg")



=== Optimal Routes ===
Route R11:
  Bins: [1, 2, 3, 8, 9, 10]
  Cost: 295.96 km
  Waste Load: 198.1 kg
------------------------------
Route R16:
  Bins: [1, 4, 7, 9]
  Cost: 175.73 km
  Waste Load: 197.1 kg
------------------------------
Route R21:
  Bins: [5, 6, 7]
  Cost: 70.64 km
  Waste Load: 139.6 kg
------------------------------
Route R23:
  Bins: [2, 3, 4, 8, 10]
  Cost: 287.20 km
  Waste Load: 197.6 kg
------------------------------
Route R30:
  Bins: [5, 6]
  Cost: 48.19 km
  Waste Load: 107.8 kg
------------------------------

Total Routes Selected: 5
Total Distance: 438.87 km
Truck Capacity Used: 200 kg
